# BibleSummary
Gives summary of chapters in the Bible.

In [14]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

bible_raw = pd.read_csv('t_web.csv', sep = ',')

# Definieer een dictionary om boeknummers naar boeknamen te mappen
book_names = {
    1: 'Genesis',
    2: 'Exodus',
    3: 'Leviticus',
    4: 'Numbers',
    5: 'Deuteronomy',
    6: 'Joshua',
    7: 'Judges',
    8: 'Ruth',
    9: '1 Samuel',
    10: '2 Samuel',
    11: '1 Kings',
    12: '2 Kings',
    13: '1 Chronicles',
    14: '2 Chronicles',
    15: 'Ezra',
    16: 'Nehemiah',
    17: 'Esther',
    18: 'Job',
    19: 'Psalms',
    20: 'Proverbs',
    21: 'Ecclesiastes',
    22: 'Song of Solomon',
    23: 'Isaiah',
    24: 'Jeremiah',
    25: 'Lamentations',
    26: 'Ezekiel',
    27: 'Daniel',
    28: 'Hosea',
    29: 'Joel',
    30: 'Amos',
    31: 'Obadiah',
    32: 'Jonah',
    33: 'Micah',
    34: 'Nahum',
    35: 'Habakkuk',
    36: 'Zephaniah',
    37: 'Haggai',
    38: 'Zechariah',
    39: 'Malachi',
    40: 'Matthew',
    41: 'Mark',
    42: 'Luke',
    43: 'John',
    44: 'Acts',
    45: 'Romans',
    46: '1 Corinthians',
    47: '2 Corinthians',
    48: 'Galatians',
    49: 'Ephesians',
    50: 'Philippians',
    51: 'Colossians',
    52: '1 Thessalonians',
    53: '2 Thessalonians',
    54: '1 Timothy',
    55: '2 Timothy',
    56: 'Titus',
    57: 'Philemon',
    58: 'Hebrews',
    59: 'James',
    60: '1 Peter',
    61: '2 Peter',
    62: '1 John',
    63: '2 John',
    64: '3 John',
    65: 'Jude',
    66: 'Revelation'
}

# Vervang de boeknummers door boeknamen in kolom 'b'
bible_raw['b'] = bible_raw['b'].map(book_names)

# Invoer voor begin- en eindvers
# begin_verse_input = 'Genesis 1 1'
# eind_verse_input = 'Genesis 2 1'

    # Functie om samenvatting te genereren
def generate_summary():
    try:
        begin_verse_input = begin_verse_entry.get()
        eind_verse_input = eind_verse_entry.get()

        # Functie om de 'id'-waarde van een vers op te halen
        def haal_id_op(boek, hoofdstuk, vers):
            query = f"{boek} {hoofdstuk} {vers}"
            result = bible_raw[bible_raw['b'] == boek][bible_raw['c'] == hoofdstuk][bible_raw['v'] == vers]['id'].values
            return result[0] if len(result) > 0 else None

        # Analyseer de invoer voor begin- en eindvers om de 'id'-waarden op te halen
        begin_boek, begin_hoofdstuk, begin_vers = begin_verse_input.split()
        eind_boek, eind_hoofdstuk, eind_vers = eind_verse_input.split()

        begin_id = haal_id_op(begin_boek, int(begin_hoofdstuk), int(begin_vers))
        eind_id = haal_id_op(eind_boek, int(eind_hoofdstuk), int(eind_vers))

        selected_rows = bible_raw[(bible_raw['id'] >= begin_id) & (bible_raw['id'] <= eind_id)]
        bible_txt = ' '.join(selected_rows['t'])

        model_name = 'google/pegasus-cnn_dailymail'
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = PegasusTokenizer.from_pretrained(model_name)
        model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

        summary_params = {
            "max_length": 200,  # De gewenste lengte van de samenvatting
            "min_length": 30,   # De gewenste minimumlengte
            "length_penalty": 2.0,  # De lengtepenaliteit aan te passen
            "num_beams": 4,  # Het aantal beams voor generatie aan te passen
            "no_repeat_ngram_size": 3,  # Herhaling van zinnen te voorkomen
        }

        # Tokenize en genereer de samenvatting
        inputs = tokenizer(bible_txt, return_tensors="pt", max_length=4096, truncation=True).to(device)
        summary_ids = model.generate(**inputs, **summary_params)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summary = summary.replace('<n>',' ')

        # Toon de samenvatting in een pop-upvenster
        messagebox.showinfo("Summary", summary)
    except Exception as e:
        # Toon een pop-upvenster met de foutmelding
        messagebox.showerror("Error", f"Fout bij het genereren van vraag.")
    
def generate_questions():
    try:
        begin_verse_input = begin_verse_entry.get()
        eind_verse_input = eind_verse_entry.get()

        # Functie om de 'id'-waarde van een vers op te halen
        def haal_id_op(boek, hoofdstuk, vers):
            query = f"{boek} {hoofdstuk} {vers}"
            result = bible_raw[bible_raw['b'] == boek][bible_raw['c'] == hoofdstuk][bible_raw['v'] == vers]['id'].values
            return result[0] if len(result) > 0 else None

        # Analyseer de invoer voor begin- en eindvers om de 'id'-waarden op te halen
        begin_boek, begin_hoofdstuk, begin_vers = begin_verse_input.split()
        eind_boek, eind_hoofdstuk, eind_vers = eind_verse_input.split()

        begin_id = haal_id_op(begin_boek, int(begin_hoofdstuk), int(begin_vers))
        eind_id = haal_id_op(eind_boek, int(eind_hoofdstuk), int(eind_vers))

        selected_rows = bible_raw[(bible_raw['id'] >= begin_id) & (bible_raw['id'] <= eind_id)]
        bible_txt = ' '.join(selected_rows['t'])

        model_name = 'voidful/context-only-question-generator'
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

        question_params = {
            "max_length": 200,  # De gewenste lengte van de samenvatting
            "min_length": 30,   # De gewenste minimumlengte
            "length_penalty": 2.0,  # De lengtepenaliteit aan te passen
            "num_beams": 4,  # Het aantal beams voor generatie aan te passen
            "no_repeat_ngram_size": 3,  # Herhaling van zinnen te voorkomen
        }

        # Tokenize en genereer de samenvatting
        inputs = tokenizer(bible_txt, return_tensors="pt", max_length=4096, truncation=True).to(device)
        question_ids = model.generate(**inputs, **question_params)
        questions = tokenizer.decode(question_ids[0], skip_special_tokens=True)
        questions = questions.replace('<n>',' ')

        # Toon de samenvatting in een pop-upvenster
        messagebox.showinfo("Question", questions)
    except Exception as e:
        # Toon een pop-upvenster met de foutmelding
        messagebox.showerror("Error", f"Fout bij het genereren van de vraag.")
    

# GUI opzetten
root = tk.Tk()
root.title("WordLightener")

# Achtergrondkleur van het rootvenster
root.configure(bg="lightblue")

# Label voor de titel
title_label = tk.Label(root, text="Welcome to", bg="lightblue", font=("Arial", 10))
title_label.pack(pady=5)

# Label voor de titel
title_label = tk.Label(root, text="WordLightener", bg="lightblue", font=("Verdana", 20, "bold"))
title_label.pack(pady=10)

# Frame voor invoervelden en knop
input_frame = tk.Frame(root, bg="lightblue")
input_frame.pack(pady=20)

# Label en invoervelden voor begin- en eindversen
begin_verse_label = tk.Label(input_frame, text="Start verse:", bg="lightblue", font=("Arial", 14))
begin_verse_label.grid(row=0, column=0, padx=10, pady=5, sticky="w")

# Pas de breedte van het invoerveld aan
begin_verse_entry = tk.Entry(input_frame, width=20, font=("Arial", 12))
begin_verse_entry.grid(row=0, column=1, padx=10, pady=5)

eind_verse_label = tk.Label(input_frame, text="End verse:", bg="lightblue", font=("Arial", 14))
eind_verse_label.grid(row=1, column=0, padx=10, pady=5, sticky="w")

# Pas de breedte van het invoerveld aan
eind_verse_entry = tk.Entry(input_frame, width=20, font=("Arial", 12))
eind_verse_entry.grid(row=1, column=1, padx=10, pady=5)

# Knop om samenvatting te genereren
generate_button = tk.Button(root, text="Generate Summary", command=generate_summary, font=("Arial", 16))
generate_button.pack(pady=10)

# Knop om vragen te genereren
generate_button_question = tk.Button(root, text="Generate Question", command=generate_questions, font=("Arial", 16))
generate_button_question.pack(pady=10)

root.geometry("400x370")  # Pas de grootte van het root window aan
root.mainloop()

C:\Users\krist\AppData\Local\Temp\ipykernel_5148\3892690438.py:95: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = bible_raw[bible_raw['b'] == boek][bible_raw['c'] == hoofdstuk][bible_raw['v'] == vers]['id'].values
C:\Users\krist\AppData\Local\Temp\ipykernel_5148\3892690438.py:141: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = bible_raw[bible_raw['b'] == boek][bible_raw['c'] == hoofdstuk][bible_raw['v'] == vers]['id'].values
